# $\pi^+$

$$A_{Siv}^{\pi^{+}}(x,z,p_{hT}) = \mathcal{A}_0(z,p_{hT},M_1)\left( \frac{\mathcal{N}_u e_u^2 f_{u}(x)D_{\pi^+/u}(z) + {\mathcal{N}_{\bar{d}} e_{\bar{d}}^2 f_{\bar{d}}(x)D_{\pi^+/{\bar{d}}}(z)}}{e_u^2 f_{u}(x)D_{\pi^+/u}(z)+e_{\bar{d}}^2 f_{\bar{d}}(x)D_{\pi^+/\bar{d}}(z)}\right)$$

## Functions

$$\mathcal{A}_0(z,p_{hT},M_1) = \frac{[z^2 \langle k_\perp^2 \rangle + \langle p_\perp^2 \rangle  ]\langle k_S^2 \rangle^2}{[z^2 \langle k_S^2 \rangle + \langle p_\perp^2 \rangle ]\langle k_\perp^2 \rangle^2} \exp \left[- \frac{p_{hT}^2z^2 \left(\langle k_S^2 \rangle - \langle k_\perp^2 \rangle \right)}{\left(z^2 \langle k_S^2 \rangle + \langle p_\perp^2 \rangle \right)\left(z^2 \langle k_\perp^2 \rangle + \langle p_\perp^2 \rangle  \right)} \right]\times \frac{\sqrt{2e}zp_{hT}}{M_1}$$

$$\mathcal{N}_q(x) = N_q x^{\alpha_q}(1-x)^{\beta_q}\frac{(\alpha_q + \beta_q)^{(\alpha_q + \beta_q)}}{\alpha_q^{\alpha_q}\beta_q^{\beta_q}}$$

$f_{u}(x)$ is the lhapdf

$D_{\pi^+/u}(z)$ is the fragmentation function

## Constants

- $\langle k_\perp^2 \rangle$ 
- $\langle p_\perp^2 \rangle$
- $\langle k_S^2 \rangle$
- $e_u$ = $2/3$
- $e_\bar{u}$ = $-2/3$
- $e_d$ = $-1/3$
- $e_\bar{d}$ = $1/3$
- $e_s$ = $-1/3$
- $e_\bar{s}$ = $1/3$
- $e$ = $1/137$

## Unknown parameters

- $N_u$
- $\alpha_u$
- $\beta_u$
- $N_{\bar{d}}$
- $\alpha_{\bar{d}}$
- $\beta_{\bar{d}}$
- $M_1$

$$\mathcal{N}_q(x) = N_q x^{\alpha_q}(1-x)^{\beta_q}\frac{(\alpha_q + \beta_q)^{(\alpha_q + \beta_q)}}{\alpha_q^{\alpha_q}\beta_q^{\beta_q}}$$

In [31]:
import lhapdf

In [48]:
class Hadron(object):
    def __init__(self, kperp2avg=.57, pperp2avg=.12, pdfset='JAM19PDF_proton_nlo',
                 ff_pion='JAM19FF_pion_nlo', ff_kaon='JAM19FF_kaon_nlo', QQ=10):
        
        self.pdfData = lhapdf.mkPDF(pdfset)
        self.ffData = lhapdf.mkPDF(ff_pion, 0)
        # needs to be extended to generalize for kaons
        self.QQ = QQ
        self.kperp2avg = kperp2avg
        self.pperp2avg = pperp2avg
        self.eu = 2/3
        self.eubar = -2/3
        self.ed = -1/3
        self.edbar = 1/3
        self.es = -1/3
        self.esbar = 1/3
        self.e = 1/137
    

    def pdf(self, flavor, x):
        return np.array([self.pdfData.xfxQ2(flavor, ax, self.QQ) for ax in x])
    
    def ff(self, flavor, z):
        return np.array([self.ffData.xfxQ2(flavor, az, self.QQ) for az in z])
    
    
    def A0(self, z, pht, m1):
        ks2avg = self.kperp2avg*m1**2/(m1**2 + self.kperp2avg)
        topfirst = (np.square(z)*self.kperp2avg + self.pperp2avg)*self.kperp2avg**2
        bottomfirst = (np.square(z)*ks2avg + self.pperp2avg) * self.kperp2avg**2
        exptop = np.square(pht) * np.square(z) * (ks2avg - self.kperp2avg)
        expbottom = (np.square(z)*ks2avg + self.pperp2avg) * (np.square(z)*self.kperp2avg + self.pperp2avg)
        last = np.sqrt(2*self.e) * z * pht / m1
        
        return (topfirst/bottomfirst) * np.exp(-exptop/expbottom) * last
    
    
    def NN(self, x, n, a, b):
        return n*np.power(x, a) * np.power(1 - x, b) * (((a + b)**(a + b))/(a**a * b**b))

    def NNanti(self, x, n):
        return n
            
    
    

In [49]:
import numpy as np

In [50]:
class PiPlus(Hadron):
    def __init__(self, kperp2avg=.57, pperp2avg=.12, pdfset='JAM19PDF_proton_nlo',
                 ff_pion='JAM19FF_pion_nlo', QQ=10):
        
        super().__init__(kperp2avg=kperp2avg, pperp2avg=pperp2avg, pdfset=pdfset,
                 ff_pion=ff_pion, QQ=QQ)
        
    
    def sivers(self, kins, Nu, au, bu, Ndbar, adbar, bdbar, m1):
        x = kins[:, 0]
        z = kins[:, 1]
        pht = kins[:, 2]
        a0 = self.A0(z, pht, m1)
        topleft = self.NN(x, Nu, au, bu) * self.eu**2 * self.pdf(2, x) * self.ff(2, z)
        topright = self.NNanti(x, Ndbar) * self.edbar**2 * self.pdf(-1, x) * self.ff(-1, z)
        bottomleft = self.eu**2 * self.pdf(2, x) * self.ff(2, z)
        bottomright = self.edbar**2 * self.pdf(-1, x) * self.ff(-1, z)
        return a0*((topleft + topright)/(bottomleft + bottomright))

In [51]:
pip = PiPlus()

In [38]:
np.array([1, 2, 3])**2

array([1, 4, 9])

In [39]:
import pandas as pd

In [40]:
df = pd.read_csv('COMPASS_d_2009.csv')

In [41]:
pipdata = df.loc[df['hadron'] == 'pi+', :]

In [43]:
kins = np.array(pipdata[['x', 'z', 'phT']])

In [ ]:
kins

In [52]:
pip.sivers(kins, 1., 1., 1., 1., 1., 1., 1.)

array([0.00307976, 0.00429643, 0.0059507 , 0.00748906, 0.01047004,
       0.01519049, 0.01977303, 0.0198893 , 0.00453345, 0.00530429,
       0.00543865, 0.00576814, 0.00688007, 0.00857435, 0.01127992,
       0.01638525, 0.02574682, 0.0018364 , 0.00305678, 0.00440506,
       0.00601993, 0.0082816 , 0.01175141, 0.01877372, 0.03707232,
       0.19386578])

In [28]:
1

0